In [32]:
import numpy as np
import pandas as pd

In [33]:
books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings = pd.read_csv('Ratings.csv')

In [34]:
books['Image-URL-M'][1]

'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [35]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [36]:
ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [37]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(179295, 8)
(1149780, 3)
(278858, 3)


In [38]:
books.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,1
Year-Of-Publication,0
Publisher,2
Image-URL-S,0
Image-URL-M,0
Image-URL-L,0


In [39]:
users.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


In [40]:
ratings.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


In [41]:
books.duplicated().sum()

np.int64(0)

In [42]:
ratings.duplicated().sum()

np.int64(0)

In [43]:
users.duplicated().sum()

np.int64(0)

Popularity Based Recommender System


In [44]:
ratings_with_name = ratings.merge(books,on='ISBN')


In [45]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Apple Magic (The Collector's series),1
2,Beyond IBM: Leadership Marketing and Finance ...,1
3,Dark Justice,1
4,Earth Prayers From around the World: 365 Pray...,10
...,...,...
161015,Ã?Â?bermorgen.,1
161016,Ã?Â?lpiraten.,2
161017,Ã?Â?rger mit Produkt X. Roman.,4
161018,Ã?Â?stlich der Berge.,3


In [46]:
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating': 'avg_rating'}, inplace=True)


In [47]:
avg_rating_df

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Apple Magic (The Collector's series),0.000000
2,Beyond IBM: Leadership Marketing and Finance ...,0.000000
3,Dark Justice,10.000000
4,Earth Prayers From around the World: 365 Pray...,5.000000
...,...,...
161015,Ã?Â?bermorgen.,0.000000
161016,Ã?Â?lpiraten.,0.000000
161017,Ã?Â?rger mit Produkt X. Roman.,5.250000
161018,Ã?Â?stlich der Berge.,2.666667


In [48]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Apple Magic (The Collector's series),1,0.000000
2,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
3,Dark Justice,1,10.000000
4,Earth Prayers From around the World: 365 Pray...,10,5.000000
...,...,...,...
161015,Ã?Â?bermorgen.,1,0.000000
161016,Ã?Â?lpiraten.,2,0.000000
161017,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
161018,Ã?Â?stlich der Berge.,3,2.666667


In [49]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)


In [50]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]


In [51]:
popular_df['Image-URL-M'][0]


'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'

Collaborative Filtering Based Recommender System


In [52]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index

In [53]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]


In [54]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [55]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]


In [56]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')


In [57]:
pt.fillna(0,inplace=True)


In [58]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271448,271705,273979,274004,274061,274301,274308,275970,277427,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
from sklearn.metrics.pairwise import cosine_similarity


In [60]:
similarity_scores = cosine_similarity(pt)


In [61]:
similarity_scores.shape


(641, 641)

In [62]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)

    return data

In [63]:
recommend('1984')


[["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg']]

In [64]:
pt.index[545]


'The Return of the King (The Lord of the Rings, Part 3)'

In [65]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [66]:
books.drop_duplicates('Book-Title')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
179289,2213613001,Bandit n'Ã?Â©tait pas manchot - Prix Quai des ...,JÃ?Â©rÃ?Â´me Jarrige,2002,Fayard,http://images.amazon.com/images/P/2213613001.0...,http://images.amazon.com/images/P/2213613001.0...,http://images.amazon.com/images/P/2213613001.0...
179290,0843932554,Green Lorelei,Francis John Thornton,1992,Leisure Books,http://images.amazon.com/images/P/0843932554.0...,http://images.amazon.com/images/P/0843932554.0...,http://images.amazon.com/images/P/0843932554.0...
179291,8431703903,Obra poÃ©tica completa (ColecciÃ³n Guernica ; 14),Miguel HernÃ¡ndez,1976,"Distribuidor exclusivo, ZYX",http://images.amazon.com/images/P/8431703903.0...,http://images.amazon.com/images/P/8431703903.0...,http://images.amazon.com/images/P/8431703903.0...
179292,2264015667,Le chat qui jouait Brahms,Lilian Jackson Braun,1999,10-18,http://images.amazon.com/images/P/2264015667.0...,http://images.amazon.com/images/P/2264015667.0...,http://images.amazon.com/images/P/2264015667.0...


In [67]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))